## 2. Report Generation

The final phase of SDMT involves aggregating evidence, validating the metrics reflected by the evidence we collected, and displaying this information in a report.

#### Preliminaries

In [15]:
# Preliminaries for loading the package locally
import os
import sys

def package_root() -> str:
    """Resolve the path to the project root."""
    return os.path.abspath(os.path.join(os.getcwd(), "..", "src/"))

sys.path.append(package_root())
sys.path.append(os.getcwd())

In [16]:
import os
from pathlib import Path

# The path at which media is stored
REPORTS_DIR = Path(os.getcwd()) / "reports"

#### Initialize MLTE Context

In [17]:
import mlte

store_path = os.path.join(os.getcwd(), "store")

mlte.set_model("IrisClassifier", "0.0.1")
mlte.set_artifact_store_uri(f"local://{store_path}")

#### Define a Binding

In MLTE, a `Binding` associates individual results with the properties to which they attest.

In [18]:
from mlte.binding import Binding

binding = Binding(
    {
        "TaskEfficacy": [
            "accuracy",
            "confusion matrix",
            "class distribution"
        ],
        "StorageCost": [
            "model size"
        ],
        "TrainingComputeCost": [
            "training cpu"
        ],
        "TrainingMemoryCost": [
            "training memory"
        ]
    }
)

# Binding also supports persistence
binding.save()

#### Validate Results

In [19]:
from mlte.measurement.result import Integer
model_size: Integer = Integer.load("model size")
model_size = model_size.less_than(3000)

In [20]:
from mlte.measurement.cpu import CPUStatistics
cpu_utilization: CPUStatistics = CPUStatistics.load("training cpu")
cpu_utilization = cpu_utilization.max_utilization_less_than(5.0)

In [21]:
from mlte.measurement.memory import MemoryStatistics
memory_consumption: MemoryStatistics = MemoryStatistics.load("training memory")
memory_consumption = memory_consumption.average_consumption_less_than(1_000)

In [22]:
from mlte.measurement.result import Real
accuracy: Real = Real.load("accuracy")
accuracy = accuracy.greater_or_equal_to(0.9)

In [23]:
from confusion_matrix import ConfusionMatrix
confusion_matrix: ConfusionMatrix = ConfusionMatrix.load("confusion matrix")
confusion_matrix = confusion_matrix.misclassification_count_less_than(2)

In [24]:
from mlte.measurement.result import Image
class_distribution: Image = Image.load("class distribution")
class_distribution = class_distribution.ignore("Inspect the image.")

### Bind `Result`s to `Property`s

In [25]:
from mlte.spec import Spec, BoundSpec
from mlte.measurement.result import Real, Integer
from mlte.measurement.memory import MemoryStatistics

# Load the specification
spec = Spec.load()

# Bind results to properties, according to Binding
bound_spec: BoundSpec = spec.bind(binding, [
    model_size,
    cpu_utilization,
    memory_consumption,
    accuracy,
    confusion_matrix,
    class_distribution
])

# BoundSpec also supports persistence
bound_spec.save()

#### Generate a Report

The final step of SDMT involves the generation of a report to communicate the results of model evaluation.

In [26]:
import time
from mlte.report import Report, Dataset, User, UseCase, Limitation

def unix_timestamp() -> str:
    return f"{int(time.time())}"

def build_report() -> Report:
    report = Report()
    report.metadata.project_name = "IrisClassificationProject"
    report.metadata.authors = ["Kyle Dotterrer", "Kate Maffey"]
    report.metadata.source_url = "https://github.com/mlte-team"
    report.metadata.artifact_url = "https://github.com/mlte-team"
    report.metadata.timestamp = unix_timestamp()

    report.model_details.name = "IrisClassifier"
    report.model_details.overview = "A model that distinguishes among three (3) types of irises."
    report.model_details.documentation = "This is a simple model that can distinguish between the setosa, versicolour, and virginica species of Iris based on physical characteristics."

    report.model_specification.domain = "Classification"
    report.model_specification.architecture = "Decision Tree"
    report.model_specification.input = "Vector[4]"
    report.model_specification.output = "Binary"
    report.model_specification.data = [
        Dataset("Dataset0", "https://github.com/mlte-team", "This is one training dataset."),
        Dataset("Dataset1", "https://github.com/mlte-team", "This is the other one we used."),
    ]

    report.considerations.users = [
        User("Botanist", "A professional botanist."),
        User("Explorer", "A weekend-warrior outdoor explorer."),
    ]
    report.considerations.use_cases = [
        UseCase("Personal Edification", "Quench your curiosity: what species of iris IS that? Wonder no longer.")
    ]
    report.considerations.limitations = [
        Limitation(
            "Low Training Data Volume",
            """
            This model was trained on a low volume of training data.
            """,
        ),
    ]
    return report

In [27]:
from mlte.report import render

# Build the base report
report = build_report()
# Attach the bound specification
report.spec = bound_spec

# Save the report as an HTML document
report.to_html(REPORTS_DIR / "report.html", local=True)